# Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import seaborn as sns
from sklearn.decomposition import FastICA

# Load audio data

In [2]:
audio_path = '/content/drive/MyDrive/PA3/Audio_Data/'

In [3]:
mic1_sr, mic1_data = wavfile.read(audio_path + 'mic1.wav')
mic2_sr, mic2_data = wavfile.read(audio_path + 'mic2.wav')
mic3_sr, mic3_data = wavfile.read(audio_path + 'mic3.wav')
mic4_sr, mic4_data = wavfile.read(audio_path + 'mic4.wav')
mic5_sr, mic5_data = wavfile.read(audio_path + 'mic5.wav')

# Form mixture matrix X

In [4]:
X = np.c_[mic1_data,mic2_data,mic3_data,mic4_data,mic5_data]

In [5]:
print(X.shape)

(191258, 5)


# ICA command

In [6]:
ica = FastICA(n_components=5)

## Experiment with:

*   Appropriate input sampling rate
*   Whether centering is required
*   Whether whitening is necessary
*   Appropriate contrastive function G(y)



### Sampling Rate

In [7]:
sr = mic1_sr
sr

22050

### Contrastive Function


### Calculate Kurtosis

In [8]:
def kurt(x):
    n = np.shape(x)[0]
    mean = np.sum((x**1)/n) 
    var = np.sum((x-mean)**2)/n 
    skew = np.sum((x-mean)**3)/n 
    kurt = np.sum((x-mean)**4)/n 
    kurt = kurt/(var**2)-3

    return kurt, skew, var, mean

In [9]:
kurtosis_X,_,_,_ = kurt(X)

In [10]:
kurtosis_X

-2.1440877455463405

### Centering

In [11]:
def center(X):
    X = np.array(X)
    mean = X.mean(axis=1, keepdims=True)
    
    return X - mean

In [12]:
X_centered = center(X)

### Whitening

In [13]:
ica = FastICA(n_components=5, random_state=1,whiten="true",fun="logcosh")
S_ = ica.fit_transform(X)
S_.shape

(191258, 5)

# Save independent components

In [14]:
from scipy.io import wavfile
volume = 50
audio_range = 32767

for ix in range(0,5):
  unmix = S_[:,ix]
  unmix_int = np.int16(unmix*audio_range*volume)
  wavfile.write(audio_path + "shat{}.wav".format(ix+1), sr, unmix_int)

In [15]:
import IPython
IPython.display.Audio(audio_path + "shat1.wav")

# Reconstruct mixture signals

In [16]:
X_recon = ica.inverse_transform(S_)

In [17]:
X_recon.shape

(191258, 5)

## Measure residuals for each one

In [18]:
residuals = []
predicted_recon_list = []
for ix in range(0,5):
  predicted_recon = X_recon[:,ix]
  predicted_recon = predicted_recon.astype('int16')
  predicted_recon_list.append(predicted_recon)
  observed_unmix = X[:,ix]
  residual_value = observed_unmix-predicted_recon
  residuals.append(np.sum(np.abs(residual_value)))

## Print out residuals

In [19]:
for ix in range(0,5):
  print("Residual for observed and predicted mic{}: {}".format(ix+1,residuals[ix]))

Residual for observed and predicted mic1: 63787
Residual for observed and predicted mic2: 67461
Residual for observed and predicted mic3: 117300
Residual for observed and predicted mic4: 47015
Residual for observed and predicted mic5: 60643


# Save reconstructed mixture signals

In [20]:
from scipy.io import wavfile
volume = 50
audio_range = 32767

for ix in range(0,5):
  recon_mix = predicted_recon_list[ix]
  wavfile.write(audio_path + "recon{}.wav".format(ix+1), sr, recon_mix)

In [21]:
import IPython
IPython.display.Audio(audio_path + "recon4.wav")